In [1]:
import numpy as np
from scipy.optimize import minimize_scalar

class DataSubjectLedger:
    """for a particular data subject, this is the list
    of all mechanisms releasing informationo about this
    particular subject, stored in a vectorized form"""
    
    def __init__(self, n_mechanisms=2):
        self.sigma = np.ones(n_mechanisms)
        self.l2_norms = np.ones(n_mechanisms) * 10
        self.l2_norms_bound = np.ones(n_mechanisms) * 40
        self.Ls = np.ones(n_mechanisms)*5
        self.coeffs = np.ones(n_mechanisms)
        self.deltas = np.zeros(n_mechanisms) + 1e-6
        self.entity = 'Bob'
        
    def get_rdp_func(self, private=True):
        
        squared_Ls = self.Ls**2
        squared_sigma = self.sigma**2
        

        
        if private:
            
            squared_L2_norms = self.l2_norms**2            
            private_constant = squared_Ls * squared_L2_norms / (2 * squared_sigma)            

            def private_rdp_func(alpha):
                return alpha * private_constant
            
            return private_rdp_func
        
        else:
            squared_L2_norm_bounds = self.l2_norms_bound**2            
            public_constant = squared_Ls * squared_L2_norm_bounds / (2 * squared_sigma)            
            def public_rdp_func(alpha):
                return alpha * public_constant
            return public_rdp_func

    def get_rdp_compose_func(self, private=True):
        
        func = self.get_rdp_func(private=private)
        
        # update the functions
        def rdp_compose_func(alpha):
            return np.dot(func(alpha), self.coeffs)
        
        return rdp_compose_func
    
    def get_alpha_search_function(self):
        rdp_compose_func = self.get_rdp_compose_func()
        delta = np.max(self.deltas)
        log_delta = np.log(delta)
        def fun(alpha):  # the input is the RDP's \alpha
            if alpha <= 1:
                return np.inf
            else:
                alpha_minus_1 = alpha-1
                return np.maximum(rdp_compose_func(alpha) + np.log(alpha_minus_1/alpha)
                                  - (log_delta + np.log(alpha))/alpha_minus_1, 0)
        return fun    
    
    def get_epsilon_spend(self, alpha_max = np.inf):
        search_fun = self.get_alpha_search_function()
        results = minimize_scalar(search_fun, method='Brent', bracket=(1,2), bounds=[1, alpha_max])

        if results.success:
            return results.fun
        else:
            # There are cases when certain \delta is not feasible.
            # For example, let p and q be uniform the privacy R.V. is either 0 or \infty and unless all \infty
            # events are taken cared of by \delta, \epsilon cannot be < \infty
            return np.inf

bob_ledger = DataSubjectLedger()

In [52]:
bob_ledger.get_epsilon_spend()

2868.0555591748666

In [46]:
fun = bob_ledger.get_alpha_search_function()

private constant:[1250. 1250.]


In [34]:
fun(1.0605677746852211)

4201.395491448622

In [35]:
fun(1.0705677746852211)

4206.719971768788

In [30]:
fun(1.0505677746852211)

4208.827647823999

     fun: 4201.395491448622
    nfev: 19
     nit: 15
 success: True
       x: 1.0605677746852211